# Работа с моделями в pytorch

In [2]:
# %matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

Мы будем использовать не только базовый функционал `pytorch`, но и библиотеку **`torchvision`**

In [3]:
!(pip freeze | grep torch) || (pip freeze | grep torch)

"grep" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [6]:
import torch
import torchvision

In [13]:
import time
text = ""
for char in tqdm(["a", "b", "c", "d"]):
    text = text + char
    time.sleep(0.5)

100%|██████████| 4/4 [00:02<00:00,  2.00it/s]


## Загрузчики данных

Зачем мы вообще ограничивали себя классом `Dataset` из модуля `torch` при работе с датасетами?

Все дело в магической силе загрузчиков из библиотеки `torchvision`. Давайте посмотрим на них в деле:

In [8]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST

Загрузчики `DataLoader` создаются поверх конкретного объекта-датасета и позволяют итерироваться по нему сразу батчами.

К этим батчам изображений-меток в свою очередь в **отдельных процессах** применяются все необходимые преобразования.

Тем самым в стандартном процессе итерирования по изображениям и рассчетам на GPU в идеале (при достаточной мощности CPU) перед каждой итерацией рассчетов на GPU уже готов очередной батч с обработанными изображениями-метками и GPU всегда утилизируется на 100%.

Рассмотрим, как это работает на примере с трансформацией картинок из MNIST:

In [9]:
transformed_mnist = MNIST("/tmp/mnist/", train=True, download=True, transform=transforms.ToTensor())

9913344it [00:03, 2875769.34it/s]                             


Extracting /tmp/mnist/MNIST\raw\train-images-idx3-ubyte.gz to /tmp/mnist/MNIST\raw



29696it [00:00, 29691073.08it/s]         


Extracting /tmp/mnist/MNIST\raw\train-labels-idx1-ubyte.gz to /tmp/mnist/MNIST\raw



1649664it [00:00, 2577453.63it/s]                             


Extracting /tmp/mnist/MNIST\raw\t10k-images-idx3-ubyte.gz to /tmp/mnist/MNIST\raw



5120it [00:00, ?it/s]                   


Extracting /tmp/mnist/MNIST\raw\t10k-labels-idx1-ubyte.gz to /tmp/mnist/MNIST\raw



In [10]:
mnist_loader = DataLoader(transformed_mnist, batch_size=16, shuffle=True, num_workers=4)

Так мы говорим загрузчику выдавать картинки и метки пачками по 16 штук, перемешав их предварительно, и использовать 4 фоновых процесса для предобработки.

"Пробежимся" теперь по всему датасету:

In [14]:
for images, targets in tqdm(mnist_loader):
    pass

100%|██████████| 3750/3750 [00:07<00:00, 509.20it/s] 


Видим, что всего вышло 3750 итераций по 16 изображений.

Посмотрим на пример батча (последнего по сути):

In [15]:
images.shape

torch.Size([16, 1, 28, 28])

In [16]:
targets.shape

torch.Size([16])

Откуда полоска `tqdm` знала, что батчей всего будет столько? Потому что в загрузчике тоже определен метод `__len__`:

In [13]:
len(mnist_loader)

3750

Посмотрим на изображение из батча (оно будет отличаться между запусками ноутбука):

In [14]:
# plt.imshow(images[0][0].numpy(), "gray")
# plt.show()

А какая метка при этом?

In [15]:
print(targets[0])

tensor(7)


Все хорошо!

## Строительный материал для нейросетей

Для определения новой питорчевой модели нужно наследовать класс `torch.nn.Module`.

Для построения же модели у нас есть строительные блоки в виде модулей из `torch.nn`.

Функциональные версии этого материала лежат в `torch.nn.functional`.

In [16]:
import torch.nn as nn
import torch.nn.functional as F

Так, у нас есть модульные и функциональные версии макспулинга, активаций, апсемплинга и другтх операций:
* `nn.MaxPool2d` / `F.max_pool2d`
* `nn.ReLU` / `F.relu`
* `nn.Upsample(mode='bilinar')` / `F.interpolate`

Для послойного объединения слоев служит класс `nn.Sequential`:

In [17]:
layers = [
    nn.Conv2d(3, 64, 3, padding=1),
    nn.ReLU(),
    nn.Conv2d(64, 64, 3, padding=1),
    nn.ReLU()
]
block = nn.Sequential(*layers)  # эквивалентно <...>(layers[0], layers[1], ..., layers[-1])
print(block)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
)


Посмотрим, как выглядит та самая модель AlexNet:

In [18]:
from torchvision.models import AlexNet

net = AlexNet()
net

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

Она состоит как раз из строительных блоков `nn.Conv2d`, `nn.ReLU`, `nn.MaxPool2d`, `nn.Dropout` и `nn.Linear`!

Слой `nn.AdaptiveAvgPool2d` служит для того, чтобы сеть можно было применить к изображению люого размера, т.к. он гарантирует, что после него карта признаков будет размера 6x6 -- дело в том, что применительно к изображениям нужного размера выход подмодуля `features` будет как размера 6x6, и если же изображения будут другого размера, то без упомянутой промежуточной операции последующее применение слоев из подмодуля приведет либо к ошибке, либо к выходу некорректного размера (а мы ожидаем 1x1x{количество классов}).

## Построение модели на примере VGG-16

Попробуем построить модель сами на примере VGG-16:

![VGG16](vgg16.png)

Сеть состоит из четырех блоков по две-три пары сверток-активаций и пулинга в конце каждого из них

Реализуем функцию для получения списка из нужных слоев для одного такого блока с нужным количеством пар:

In [19]:
def features_block(in_channels, out_channels, num_pairs):
    block = []
    block.append(nn.Conv2d(in_channels, out_channels, 3, padding=1))
    block.append(nn.ReLU(inplace=True))
    for i in range(num_pairs - 1):
        block.append(nn.Conv2d(out_channels, out_channels, 3, padding=1))
        block.append(nn.ReLU(inplace=True))
    block.append(nn.MaxPool2d(kernel_size=2))
    return block

А также функцию, выдающаю весь список сверток-активаций-пулингов до полносвязных слоев:

In [20]:
def features(in_channels, out_channels_list, block_sizes):
    layers = []
    for out_channels, block_size in zip(out_channels_list, block_sizes):
        layers.extend(features_block(in_channels, out_channels, block_size))
        in_channels = out_channels
    return layers

Нам также необходима "классификационная" часть сети из нескольких полносвязных слоев:

In [21]:
def classifier():
    layers = []
    layers.append(nn.Linear(512 * 7 * 7, 4096))
    layers.append(nn.ReLU(inplace=True))
    layers.append(nn.Dropout2d(p=0.5))
    layers.append(nn.Linear(4096, 4096))
    layers.append(nn.ReLU(inplace=True))
    layers.append(nn.Dropout2d(p=0.5))
    layers.append(nn.Linear(4096, 1000))
    return layers

Вся нейросеть представляет из себя последовательное выделение признаков и применение полносвязных слоев:

In [22]:
class VGG16(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(*features(3, [64, 128, 256, 512, 512], [2, 2, 3, 3, 3]))
        self.classifier = nn.Sequential(*classifier())

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

Создадим нейросеть и посмотрим на нее:

In [23]:
net = VGG16()

net

VGG16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

## Использование предобученных сетей

Давайте сравним созданную нами модель с готовой моделью из `torchvision`:

In [24]:
from torchvision.models import vgg16

vgg16()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Как видим, разницы практически нет! Единственное отличие -- промежуточный модуль `nn.AdaptiveAvgPool2d`, который мы уже рассматривали.

У каждой модели есть обучаемые веса, которые можно получить в виде словаря "состояния" сети:

In [25]:
net.state_dict()

,  9.1462e-03,
                       7.6926e-03, -1.0356e-02,  1.0496e-02, -1.0014e-02,  4.4989e-03,
                       4.8017e-03,  1.3897e-02,  5.9299e-03, -1.2475e-02,  4.6709e-03,
                      -9.5180e-03, -8.8206e-03,  5.5096e-03, -5.5526e-03,  8.7798e-03,
                       1.1100e-02, -1.4487e-02])),
             ('classifier.0.weight',
              tensor([[ 3.6846e-03,  3.4358e-03, -3.3585e-03,  ...,  3.9722e-03,
                        1.9455e-03,  4.4120e-03],
                      [ 3.9170e-03, -9.2679e-04,  1.2423e-03,  ...,  5.3054e-03,
                        1.5218e-03,  6.1396e-03],
                      [-1.5370e-03,  1.2610e-03,  6.0562e-03,  ...,  5.7767e-03,
                       -4.3377e-03, -3.3553e-03],
                      ...,
                      [ 2.6700e-03,  5.4982e-03, -2.1279e-03,  ...,  1.6752e-03,
                        1.3511e-03, -5.5768e-03],
                      [-2.6822e-03, -3.7450e-03, -4.1084e-03,  ..., -3.0506e-03,
    

В нашем случае там пока что находятся случайно инициализированные веса.

Но в библиотеке `torchvision` для многих моделей есть и предобученные на датасете ImageNet веса.

Так, предобученные веса для модели VGG-16 можно получить, указав параметр `pretrained=True` при создании:

In [26]:
vgg16(pretrained=True).state_dict()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/eos/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [02:50<00:00, 3.25MB/s]


03e-02, -1.0054e-02,
                       2.1246e-01,  1.3612e-01, -7.6917e-01,  4.1772e-02,  5.9762e-02,
                       1.0543e-03, -1.5168e-02,  2.7847e-01,  5.2689e-02,  1.2394e-01,
                       3.0762e-02,  1.4299e-01,  1.2184e-02,  3.3403e-02, -1.4907e-02,
                      -6.7975e-02,  3.4594e-01])),
             ('classifier.0.weight',
              tensor([[-0.0011, -0.0027,  0.0022,  ...,  0.0066, -0.0004, -0.0021],
                      [ 0.0052,  0.0020,  0.0046,  ..., -0.0054, -0.0045, -0.0019],
                      [-0.0005,  0.0052,  0.0018,  ...,  0.0068,  0.0005,  0.0091],
                      ...,
                      [-0.0075, -0.0096, -0.0025,  ..., -0.0079, -0.0106, -0.0036],
                      [-0.0004,  0.0014, -0.0019,  ...,  0.0036,  0.0021,  0.0038],
                      [ 0.0063,  0.0041, -0.0004,  ..., -0.0030,  0.0011,  0.0047]])),
             ('classifier.0.bias',
              tensor([ 0.0341,  0.0021,  0.0217,  ..., -0.006

Словарь можно не только получить для обученной модели, но и наоборот загрузить его как веса в модель!

Раз мы сами создали практически идентичную стандартной модель, то может веса подойдут для нее?

In [27]:
net.load_state_dict(vgg16(pretrained=True).state_dict())

<All keys matched successfully>

Успех!

## Предсказание построенной моделью

Загрузим изображение

In [28]:
from PIL import Image

image = Image.open('cat.jpg')

image

FileNotFoundError: [Errno 2] No such file or directory: 'cat.jpg'

Создадим стандартный iamgenet-трансформер для предобработки

In [29]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Сформируем батч размера 1 из тензора с предобработанным изображением

In [30]:
tensor = transform(image)
batch = torch.stack([tensor])

NameError: name 'image' is not defined

Произведем инференс модели

In [31]:
with torch.no_grad():
    output = net(batch)

NameError: name 'batch' is not defined

На выходе получем сырой вектор из 1000 т.н. логитов (до применения операции softmax)

In [32]:
output.shape

NameError: name 'output' is not defined

Какой класс наиболее вероятен?

In [33]:
output.argmax()

NameError: name 'output' is not defined

А просто число можно?

In [34]:
output.argmax().item()

NameError: name 'output' is not defined

Загрузим названия классов для ImageNet 1k:

In [35]:
import json

with open('labels.dict') as fin:
    labels = eval(fin.read())

FileNotFoundError: [Errno 2] No such file or directory: 'labels.dict'

Так все-таки что там на картинке?

In [36]:
labels[output.argmax().item()]

NameError: name 'labels' is not defined

Ура!

## Домашнее задание

Реализуйте аналогично класс для модели LeNet-5

![LeNet-5](lenet.png)

Создайте экземпляр нейросети и примените его к нескольким примерам из MNIST.

Убедитесь, что сеть выдает случайные классы, т.к. веса были проинициализированы случайно.